In [1]:
import os
import pandas as pd
import numpy as np
from haversine import haversine, Unit
import xgboost
import pickle as pk
from sklearn import preprocessing
from sklearn import pipeline
from sklearn import model_selection
from sklearn import linear_model
from sklearn import tree
from sklearn import svm
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn import neural_network
from sklearn import ensemble
from sklearn import metrics

from data import feature_engineering

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(context='paper', font_scale=1.15)

data_dir = '../data/datathon_SC_ACN_22/'

c:\Users\linus\anaconda3\envs\datathon-fme2022\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### Data

In [2]:
orders_data = pd.read_csv(data_dir + 'test.csv', delimiter=';', index_col='order_id')

cities_data = pd.read_csv(data_dir + 'cities_data.csv', delimiter=';')
product_data = pd.read_csv(data_dir + 'product_attributes.csv', delimiter=',', index_col='product_id')

In [3]:
orders_features = feature_engineering.engineer_features(orders_data, cities_data, product_data)

### Load model

In [4]:
with open('../models/ensemble_classifier_v001.pkl', 'rb') as f:
    model = pk.load(f)

In [11]:
predictions = model.predict(orders_features)
orders_data_predicted = orders_data.reset_index().join(pd.Series(predictions).rename('prediction')).set_index('product_id')

orders_data_predicted.sample(n=10)

,order_id,origin_port,3pl,customs_procedures,logistic_hub,customer,units,prediction
product_id,,,,,,,,
1675746,94baa6c9237c,Athens,v_002,DTD,Hamburg,Berlin,476,0
1696488,73c81c616ceb,Rotterdam,v_002,DTP,Venlo,Porto,353,0
1692723,ecf327ac9f11,Rotterdam,v_004,CRF,Hamburg,Helsinki,537,0
1657013,ac7c82542bc0,Barcelona,v_004,CRF,Bratislava,Paris,570,0
1685076,2f842a6a5681,Rotterdam,v_002,DTD,Dusseldorf,Berlin,453,0
1653969,93e4fac11d8f,Barcelona,v_002,DTD,Rome,Munich,463,0
1687521,f6aedefae242,Rotterdam,v_002,DTD,Warsaw,Malmö,532,0
1689549,b19b33a665f6,Barcelona,v_001,DTD,Dusseldorf,Madrid,481,0
1699416,3992b329c12a,Rotterdam,v_002,DTD,Venlo,Athens,466,0
